# KNMI WEATHER DATA

Install the unofficial KNMI-package: https://pypi.org/project/knmi-py/

Functions
- knmi.get_day_data_raw(stations, start, end, inseason, variables) 
- knmi.get_day_data_dataframe(stations, start, end, inseason, variables) 
- knmi.get_hour_data_raw(stations, start, end, inseason, variables) 
- knmi.get_hour_data_dataframe(stations, start, end, inseason, variables) 
- knmi.get_forecast_dataframe(station, conform_values, variables)


- knmi.stations provides a list of all available stations
- knmi.variables provides a list of all variables and explanations

In [47]:
import pandas as pd
import regex as re
import knmi
import requests

In [12]:
# check out the stations in the knmi-package 
knmi.stations

{8: Station(number=8, longitude=5.528, latitude=52.453, altitude=-4.2, name='LELYSTAD'),
 13: Station(number=13, longitude=3.314, latitude=47.975, altitude=999.9, name='WISSEKERKE ???'),
 16: Station(number=16, longitude=3.314, latitude=47.975, altitude=999.9, name='HEINKENSZAND ???'),
 17: Station(number=17, longitude=3.892, latitude=51.529, altitude=0.7, name='WILHELMINADORP'),
 18: Station(number=18, longitude=3.314, latitude=47.975, altitude=999.9, name='PHILIPPINE ???'),
 19: Station(number=19, longitude=3.314, latitude=47.975, altitude=999.9, name='NIEUWE TONGE ???'),
 20: Station(number=20, longitude=6.7, latitude=51.969, altitude=34.1, name='WINTERSWIJK'),
 28: Station(number=28, longitude=5.82, latitude=52.961, altitude=-0.4, name='JOURE'),
 29: Station(number=29, longitude=5.044, latitude=52.65, altitude=999.9, name='HOORN (NH)'),
 32: Station(number=32, longitude=4.526, latitude=51.576, altitude=1.2, name='OUDENBOSCH'),
 33: Station(number=33, longitude=5.677, latitude=51.54

In [31]:
knmi_stations = knmi.stations

df_knmi_stations = pd.DataFrame(knmi_stations).transpose()
df_knmi_stations.columns=["number", "longitude", "latitude", "altitude", "name"]

pd.set_option('display.max_rows', df_knmi_stations.shape[0]+1)
df_knmi_stations.sort_values("name", ascending=True)

,number,longitude,latitude,altitude,name
205,205,3.81,55.401,48.4,A12-CPP
122,122,6.303,52.158,9,ALMEN
616,616,4.907,52.367,999.9,AMSTERDAM (COENHAVEN)
125,125,5.054,51.784,1.4,ANDEL
617,617,6.196,51.498,19,ARCEN
391,391,6.197,51.498,19.5,ARCEN
551,551,2.066,56.4,999.9,AUK
253,253,2.067,56.4,999.9,AUK-ALFA
208,208,5.942,53.493,40.5,AWG-1
249,249,4.979,52.644,-2.4,BERKHOUT


In [28]:
#locate the stations that belongs to each check-point
station_needed = {"Stretch_1": "De Bilt", "Stretch_2": "Twenthe", "Stretch_3": "Eelde", "Stretch_4": "Lauwersoog", "Stretch_5": "Lelystad","Stretch_6": "Den Helder", "Stretch_7":"Hoek van Holland", "Stretch_8":"Vlissingen"  , "Stretch_9": "Ell", "Stretch_10_finish": "Arcen"} 
df_station_needed = pd.DataFrame.from_dict(station_needed, orient = 'index').reset_index()
df_station_needed.columns = [ "stretch", "station_needed"]
df_station_needed



,stretch,station_needed
0,Stretch_1,De Bilt
1,Stretch_2,Twente
2,Stretch_3,Eelde
3,Stretch_4,Lauwersoog
4,Stretch_5,Lelystad
5,Stretch_6,Den Helder
6,Stretch_7,Hoek van Holland
7,Stretch_8,Vlissingen
8,Stretch_9,Ell
9,Stretch_10_finish,Arcen


In [5]:
#check out the variables in the KNMI-package 
knmi.variables

{'FXXH': 'Hourly division in which FXX was measured',
 'PXH': 'Hourly division in which PX was measured',
 'TG': 'Daily mean temperature in (0.1 degrees Celsius)',
 'RHXH': 'Hourly division in which RHX was measured',
 'EV24': 'Potential evapotranspiration (Makkink) (in 0.1 mm)',
 'UNH': 'Hourly division in which UN was measured',
 'UX': 'Maximum relative atmospheric humidity (in percents)',
 'TN': 'Minimum temperature (in 0.1 degrees Celsius)',
 'FHXH': 'Hourly division in which FHX was measured',
 'PG': 'Daily mean sea level pressure (in 0.1 hPa) calculated from 24 hourly values',
 'FG': 'Daily mean windspeed (in 0.1 m/s)',
 'SQ': 'Sunshine duration (in 0.1 hour) calculated from global radiation (-1 for <0.05 hour)',
 'FHNH': 'Hourly division in which FHN was measured',
 'T10NH': '6-hourly division in which T10N was measured; 6=0-6 UT, 12=6-12 UT, 18=12-18 UT, 24=18-24 UT',
 'VVN': 'Minimum visibility; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:

In [34]:
knmi_variables = knmi.variables

df_knmi_variables = pd.DataFrame.from_dict(knmi_variables, orient = 'index').reset_index()
df_knmi_variables.columns = [ "Shorthand", "Meaning"]
df_knmi_variables

#I would be interested in: date; daily rainfall (mm); avg windspeed; direction of the wind
# YYYYMMDD 
# RH -> Daily precipitation amount (in 0.1 mm) (-1 for...
# TG -> Daily mean temperature in (0.1 degrees Celsius)
# FG -> Daily mean windspeed (in 0.1 m/s)
# FHVEC -> Vector mean windspeed (in 0.1 m/s)
# DDVEC -> Vector mean wind direction in degrees (360=nor...

,Shorthand,Meaning
0,FXXH,Hourly division in which FXX was measured
1,PXH,Hourly division in which PX was measured
2,TG,Daily mean temperature in (0.1 degrees Celsius)
3,RHXH,Hourly division in which RHX was measured
4,EV24,Potential evapotranspiration (Makkink) (in 0.1...
5,UNH,Hourly division in which UN was measured
6,UX,Maximum relative atmospheric humidity (in perc...
7,TN,Minimum temperature (in 0.1 degrees Celsius)
8,FHXH,Hourly division in which FHX was measured
9,PG,Daily mean sea level pressure (in 0.1 hPa) cal...


In [61]:
df = knmi.get_day_data_dataframe(stations=[260])
print(df.disclaimer) 
print(df.stations) 
print(df.legend) 
df = df.rename(columns=df.legend) 
print(df)

StopIteration: 

In [49]:
knmi.stations
knmi.get_day_data_raw(stations, start, end, inseason, variables)

NameError: name 'stations' is not defined

In [55]:
from knmy import knmy

In [56]:
knmy.get_daily_data(stations=[209], start=20170101, end=20170106, variables=['WIND','TEMP', 'SQ'])

'<!DOCTYPE html>\n<!--[if lte IE 8]>\n  <html class="no-js ie" lang="nl">\n<![endif]-->\n<!--[if gt IE 8]><!-->\n  <html class="no-js" lang="nl">\n<!--<![endif]-->\n<head>\n  <meta charset="utf-8" />\n  <title>KNMI - Klimatologie</title>\n  <meta name="description" content="">\n  <meta name="author" content="" />\n  <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n  <link rel="shortcut icon" href="https://cdn.knmi.nl/assets/favicon-06dd94e551015e92293dd6d7525ab5c1ab9a51f68bebdf0b4669c12d69dbe522.ico" />\n  <link rel="apple-touch-icon" href="https://cdn.knmi.nl/assets/apple-touch-icon-220a47425fd62d44dc04800d3067032801342cd842cc7935b89dbca0cec4a7c9.png" />\n  <link rel="stylesheet" media="all" href="https://cdn.knmi.nl/assets/application-6931331cf59bf7a56417431ea109a5e2dbcd9d116736830f5e872278fd229e36.css" data-turbolinks-track="true" />\n  <script src="https://cdn.knmi.nl/assets/modernizr/modernizr.min-61c5cef44c1ccbcd872b5a6eacda768c609564a4b1008880c84b85477a0

In [59]:
disclaimer, stations, variables, data = knmy.get_hourly_data(stations=[209, 235], start=2015010108, end=2017010620,
                                                             inseason=True, variables=['WIND', 'TEMP'], parse=True)

StopIteration: 

In [60]:
# disclaimer, variables, data = knmy.get_daily_rain_data(start=20170101, end=20170101, parse=True)